In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_getting-started-movielens-01-download-convert/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started MovieLens: Download and Convert

This notebook is created using the latest stable [merlin-hugectr](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-hugectr/tags), [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags), or [merlin-pytorch](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-pytorch/tags) container.

## MovieLens25M

The [MovieLens25M](https://grouplens.org/datasets/movielens/25m/) is a popular dataset for recommender systems and is used in academic publications. The dataset contains 25M movie ratings for 62,000 movies given by 162,000 users. Many projects use only the user/item/rating information of MovieLens, but the original dataset provides metadata for the movies, as well. For example, which genres a movie has. Although we may not improve state-of-the-art results with our neural network architecture in this example, we will use the metadata to show how to multi-hot encode the categorical features.

## Download the dataset

In [2]:
# External dependencies
import os
import pandas as pd
from merlin.core.utils import download_file


# Get dataframe library - cudf or pandas
from merlin.core.dispatch import get_lib
df_lib = get_lib()

We define our base input directory, containing the data.

In [3]:
import dotenv
dotenv.load_dotenv('.env')

True

In [4]:
# DATASET_NAME = "ml-1m"
# DATASET_NAME = "ml-25m"
DATASET_NAME = os.getenv("DATASET_NAME")

In [5]:
# INPUT_DATA_DIR = "/Users/nelsonlin/Desktop/data/movielens"
INPUT_DATA_DIR = os.getenv("INPUT_DATA_DIR")

We will download and unzip the data.

In [6]:
download_unzip_path = os.path.join(INPUT_DATA_DIR,DATASET_NAME)

In [7]:
download_unzip_path

'/Users/nelsonlin/Desktop/data/movielens/ml-25m'

In [8]:
if not os.path.exists(download_unzip_path):
    download_file(
        f"http://files.grouplens.org/datasets/movielens/{DATASET_NAME}.zip",
        os.path.join(INPUT_DATA_DIR, f"{DATASET_NAME}.zip"),
    )

## Convert the dataset

First, we take a look on the movie metadata. 

In [9]:
INPUT_DATA_DIR

'/Users/nelsonlin/Desktop/data/movielens'

In [10]:
if DATASET_NAME == 'ml-1m':
    movies = pd.read_csv(os.path.join(INPUT_DATA_DIR,DATASET_NAME,"movies.dat"),
                            delimiter='::', engine= 'python', header=None,
                            names=['title', 'genres'],encoding="iso-8859-1")


In [11]:
if DATASET_NAME == 'ml-25m':
    movies = df_lib.read_csv(os.path.join(INPUT_DATA_DIR, f"{DATASET_NAME}/movies.csv"))

We can see, that genres are a multi-hot categorical features with different number of genres per movie. Currently, genres is a String and we want split the String into a list of Strings. In addition, we drop the title.

In [12]:
movies["genres"] = movies["genres"].str.split("|")
movies = movies.drop("title", axis=1)
movies.head()

movieId                                             genres
0        1  [Adventure, Animation, Children, Comedy, Fantasy]
1        2                     [Adventure, Children, Fantasy]
2        3                                  [Comedy, Romance]
3        4                           [Comedy, Drama, Romance]
4        5                                           [Comedy]

We save movies genres in parquet format, so that they can be used by NVTabular in the next notebook.

In [13]:
movies.to_parquet(os.path.join(INPUT_DATA_DIR,DATASET_NAME,"movies_converted.parquet"))

## Splitting into train and validation dataset

We load the movie ratings.

In [14]:
if DATASET_NAME == 'ml-1m':
    ratings = pd.read_csv(os.path.join(INPUT_DATA_DIR,DATASET_NAME,"ratings.dat"),
                            delimiter='::', engine= 'python', header=None,
                            names=['userId', 'movieId', 'rating', 'timestamp'],encoding="iso-8859-1")

In [15]:
if DATASET_NAME == 'ml-25m':
    ratings = df_lib.read_csv(os.path.join(INPUT_DATA_DIR, DATASET_NAME, "ratings.csv"))

In [16]:
ratings.head()

userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510

We drop the timestamp column and split the ratings into training and test datasets. We use a simple random split.

In [17]:
ratings = ratings.drop("timestamp", axis=1)

# shuffle the dataset
ratings = ratings.sample(len(ratings), replace=False)

# split the train_df as training and validation data sets.
num_valid = int(len(ratings) * 0.2)

train = ratings[:-num_valid]
valid = ratings[-num_valid:]

In [19]:
len(train)

20000076

We save the dataset to disk.

In [18]:
train.to_parquet(os.path.join(INPUT_DATA_DIR,DATASET_NAME,"train.parquet"))
valid.to_parquet(os.path.join(INPUT_DATA_DIR,DATASET_NAME, "valid.parquet"))